In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import torch
from torch import nn
from torch.nn import functional as F
from torchmetrics.classification import MulticlassAccuracy
import torchvision

from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

%env "WANDB_NOTEBOOK_NAME" "civar10_template"
import wandb

c:\Users\koeni\anaconda3\envs\ml\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
c:\Users\koeni\anaconda3\envs\ml\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:92: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[FeatureMapContrastiv

env: "WANDB_NOTEBOOK_NAME"="civar10_template"


In [2]:
PATH_DATASETS = '..\data'

In [3]:
class CIFAR10Model(pl.LightningModule):
    def __init__(self, lr=0.1, mean=0., sd=1.):
        super().__init__()

        # initial args
        self.lr = lr
        self.metric = MulticlassAccuracy(num_classes=10)
        self.mean = torch.tensor(mean)
        self.sd = torch.tensor(sd)

        # define model
        self.model = nn.Sequential(
            nn.Conv2d(3, 6, 3),
            nn.MaxPool2d(2, 2), 
            nn.ReLU(),
            nn.Conv2d(6, 12, 4),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.Conv2d(12, 24, 3),
            nn.ReLU(),
            nn.Flatten(1),
            nn.Linear(24*4*4, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

        self.save_hyperparameters()

    def forward(self, x):
        return self.model(x)

    def _common_step(self, batch, batch_idx, stage :str):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)

        with torch.no_grad():
            acc = self.metric(logits, y)

            self.log(f'{stage}/loss', loss)
            self.log(f'{stage}/accuracy', acc)

        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self._common_step(batch, batch_idx, 'train')
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss, _ = self._common_step(batch, batch_idx, 'val')

    def test_step(self, batch, batch_idx):
        loss, logits = self._common_step(batch, batch_idx, 'test')
        x, y = batch

        images = []
        captions = []
        mean = self.mean.view(1, 3, 1, 1).to(x.device)
        sd = self.sd.view(1, 3, 1, 1).to(x.device)
        for i in range(1):
            img = x[i]*sd + mean
            pred = F.softmax(logits[i], dim=0)
            images.append(img)
            captions.append('predictions: ' + str(pred.detach().cpu().numpy().round(2)))

        self.logger.log_image(key='test_predictions', images=images, caption=captions)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [4]:
wandb.login()
logger = WandbLogger(
    project="cifar10_template",
    name='conv',
    log_model=True,
    save_dir='../logs',
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koero. Use `wandb login --relogin` to force relogin


In [5]:
train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

cifar = CIFAR10DataModule(
    data_dir=PATH_DATASETS,
    batch_size=256,
    num_workers=1,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    val_transforms=test_transforms,
)
cifar.prepare_data()
cifar.setup()

Files already downloaded and verified
Files already downloaded and verified


In [6]:
cn = cifar10_normalization()
model = CIFAR10Model(
    lr=0.0001, 
    mean=cn.mean, 
    sd=cn.std
)

In [7]:
trainer = Trainer(
    max_epochs=10,
    accelerator='gpu',
    accumulate_grad_batches=1,
    log_every_n_steps=10,
    logger=logger
)
trainer.fit(
    model = model,
    datamodule=cifar
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params
----------------------------------------------
0 | metric | MulticlassAccuracy | 0     
1 | model  | Sequential         | 62.1 K
----------------------------------------------
62.1 K    Trainable params
0         Non-trainable params
62.1 K    Total params
0.249     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\koeni\anaconda3\envs\ml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\koeni\anaconda3\envs\ml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [8]:
trainer.test(
    model = model,
    datamodule=cifar
)

Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\koeni\anaconda3\envs\ml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │    0.41044116020202637    │
│         test/loss         │     1.632372498512268     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 1.632372498512268, 'test/accuracy': 0.41044116020202637}]

In [9]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█
test/accuracy,▁
test/loss,▁
train/accuracy,▁▁▂▃▃▄▅▅▄▄▅▆▅▅▅▅▅▇▆▅▅▆▆▇▇▇▆▇▆▇▆█▇██▇▇▆█▇
train/loss,███▇▇▆▅▅▅▅▅▄▄▄▄▄▄▃▄▃▄▄▃▃▃▃▄▂▃▁▃▂▂▁▂▃▂▂▁▃
trainer/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇██████████
val/accuracy,▁▃▄▅▅▆▇▇██
val/loss,█▆▅▄▃▃▂▂▁▁
epoch,10
test/accuracy,0.41044
test/loss,1.63237
